Project
======


In [2]:
# This line is a convenience to import most packages you'll need. You may need to import others (eg random and cmath)
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math
from IPython.display import Audio
 
# This line makes sure your plots happen IN the webpage you're building, instead of in separate windows.
%matplotlib inline

D:\Program Files (x86)\Miniconda2\envs\eecs352\lib\site-packages\librosa\core\audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '
D:\Program Files (x86)\Miniconda2\envs\eecs352\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
from scipy.fftpack import fft
from scipy.signal import hann

def wavwrite(filepath, data, sr, norm=True, dtype='int16',):
    '''
    Write wave file using scipy.io.wavefile.write, converting from a float (-1.0 : 1.0) numpy array to an integer array
    
    Parameters
    ----------
    filepath : str
        The path of the output .wav file
    data : np.array
        The float-type audio array
    sr : int
        The sampling rate
    norm : bool
        If True, normalize the audio to -1.0 to 1.0 before converting integer
    dtype : str
        The output type. Typically leave this at the default of 'int16'.
    '''
    if norm:
        data /= np.max(np.abs(data))
    data = data * np.iinfo(dtype).max
    data = data.astype(dtype)
    sp.io.wavfile.write(filepath, sr, data)
    
#-----------------------------------------------------------
# Here's an stft implementation. Yes, it doesn't bother to get the last little bit of audio.
   
def stft(signal, window_size, hop_size, window_type = 'hann'):
    """
    Computes the short term fourier transform of a 1-D numpy array, where the array 
    is windowed into a set of subarrays, each of length window_size. The distance between
    window centers (in samples) is given by hop_size. The type of window applied is
    determined by window_type. This returns a 2-D numpy array where the ith column
    is the FFT of the ith window. Each column contains an array of complex values.
    
    Input Parameters
    ----------------
    signal: The 1-d (complex or real) numpy array containing the signal
    window_size: an integer scalar specifying the number of samples in a window
    hop_size: an integer specifying the number of samples between the start of adjacent windows
    window_type: a string specifying one of two "hann" or "rectangular"
    
    Returns
    -------
    a 2D numpy array of complex numbers where the array column is the FFT of the ith window,
    and the jth element in the ith column is the jth frequency of analysis.
    """
    # figure out how many hops
    length_to_cover_with_hops = len(signal) - window_size;
    assert (length_to_cover_with_hops >= 0), "window_size cannot be longer than the signal to be windowed"
    num_hops = 1 + length_to_cover_with_hops/hop_size;
    
    # make our window function
    if window_type == 'hann':
        window = hann(window_size)
    elif window_type=='rectangular':
        window = np.ones(window_size)
    else:
        raise Exception('Invalid window type. Must be "hann" or "rectangular".')
    
    stft = [0]*num_hops
    # fill the array with values 
    for hop in range(num_hops):
        start = hop*hop_size
        end = start + window_size
        unwindowed_sound = signal[start:end]
        windowed_sound =  unwindowed_sound * window
        stft[hop]= fft(windowed_sound, window_size) 
    return np.array(stft)

#-----------------------------------------------------------
# here's a spectrogram implementation

def spectrogram(signal, window_size, hop_size, sample_rate, window_type = 'hann', display = True, tick_labels ='frame-freqindex' ):
    """
    Computes the short term fourier transform of a 1-D numpy array, where the array 
    is windowed into a set of subarrays, each of length window_size. The distance between
    window centers (in samples) is given by hop_size. The type of window applied is
    determined by window_type. This creates a 2-D numpy array where the ith column
    is the FFT of the ith window. Each column contains an array of complex values.
    It then creates a magnitude spectrogram of the signal and plots it on the screen.
    Here, the vertical dimension is frequency (in Hz), the horizontal dimension is time
    (in seconds), brightness corresponds to amplitude (in dB). Only frequencies up to
    the Nyquist rate are displayed.
    
    Input Parameters
    ----------------
    signal: The 1-d (complex or real) numpy array containing the signal
    window_size: an integer scalar specifying the number of samples in a window
    hop_size: an integer specifying the number of samples between the start of adjacent windows
    sample_rate: an integer giving the sample rate of the input signal, in Hz
    window_type: a string specifying one of two "hann" or "rectangular"
    display: a bool. If set to True, it plots the spectrogram. Else it does not.
    
    Returns
    -------
    an output tuple with 3 items
    
    sgram:  a 2-D numpy array of real-valued numbers that contains the magnitude spectrogram
           sgram[t,f] is the magnitude at time t and frequency f. This only contains values
           up to the nyquist frequency
    times: a 1-D numpy array of non-negative real-values that gives the times,  
           times[t] gives the start time of the tth window in seconds
    freqs: a 1-D numpy array  of non-negative real values. freqs[f] gives the fth
           frequency of analysis in Hz, up to the nyquist frequency
           
    Calling Example
    ---------------
    sgram,times,freqs = spectrogram(signal, window_size, hop_size, sample_rate )

    """ 

    # get the stft
    X = stft(signal, window_size, hop_size, window_type)

    # figure out what my window start times are
    times = np.arange(len(X))
    hop_in_secs = hop_size/(1.0 * sample_rate)
    times = times * hop_in_secs
    
    # chop off everything above the nyquist rate & rotate for display    
    X = np.rot90(X,3)
    sgram = np.array(X[0:len(X)/2])

    # figure out what my frequencies of analysis are..
    analysis_f0 = sample_rate/window_size
    freqs = np.arange(len(sgram)) * analysis_f0
    
    # turn the complex values into magnitudes and putit into a log scale 
    sgram = np.abs(sgram) + 0.00000000000001  # this prevents taking the log of 0
    log_sgram = 20 * np.log10(sgram)
 
    # plot the spectrum 
            
    if display: 
        plt.figure()
        if tick_labels == 'frame-freqindex':
            times2 = np.arange(len(times))
            freqs2 = np.arange(len(freqs))
            x_coord = np.tile(times2,(len(freqs2),1)) 
            x_coord = np.fliplr(x_coord)
            y_coord = np.tile(freqs2,(len(times2),1))
            y_coord = np.rot90(y_coord,3)

            plt.pcolor(x_coord,y_coord,log_sgram)
            # set the limits of the plot to the limits of the data
            plt.axis([x_coord.min(), x_coord.max(), y_coord.min(), y_coord.max()])

            plt.title('Magnitude Spectrogram in dB')
            plt.xlabel('index of time')
            plt.ylabel('index of frequency')
            cbar = plt.colorbar()
            
        elif tick_labels == 'time-freq':
            x_coord = np.tile(times,(len(freqs),1)) 
            x_coord = np.fliplr(x_coord)
            y_coord = np.tile(freqs,(len(times),1))
            y_coord = np.rot90(y_coord,3)

            plt.pcolor(x_coord,y_coord,log_sgram)
            # set the limits of the plot to the limits of the data
            plt.axis([x_coord.min(), x_coord.max(), y_coord.min(), y_coord.max()])

            plt.title('Magnitude Spectrogram in dB')
            plt.xlabel('time in seconds')
            plt.ylabel('frequency in Hz')
            cbar = plt.colorbar()
    
    # return from dB to amplitude
    sgram = 10**(np.rot90(log_sgram,1)/20.0)
    return sgram, times, freqs
#--------------------------------------
# A couple of helper functions to make mels into frequencies and vice versa

def mel2freq(mel):
    freq = 700.0 * (-1.0 + 10.0**(mel/2595.0))
    return freq

def freq2mel(freq):
    x = 1.0 + freq/700.0
    mel = 2595 * np.log10(x)
    return mel

#-----------------------------------------------------------
# Exactly what it says it is...

def make_triangular_filters(cfreqs, freqs ):
    """
    Computes a set of "filters" to apply to an STFT to change its frequency scaling.
    This takes an array of center frequencies (cfreqs) and a set of frequencies of 
    analysis (freqs) as input, both coded in Hz.  It will return a 2-D numpy array 
    filters[c][f], where c is the index number of a center frequency in cfreqs and 
    f is the index to a frequency in freqs and the value in filters[c][f] is a number 
    between 0 and 1 that represents how sensitive this filter is to that frequency.  
    Filters are triangular, with a linear decrease in sensitivity from the center 
    frequency (sensitivity = 1) down to a sensitivity of 0 by the center frequency 
    of each adjacent freqency.
    
    Input Parameters
    ----------------
    cfreqs: a 1-D python array of center frequencies measured in Hz
    freqs:   a 1-D python array of frequencies for whom we need to calculate the sensitivity
             of each filter. 
    
    Returns
    -------
    filters: a 2-D numpy array, where c is the index number of a center frequency in 
            cfreqs and f is the index to a frequency in freqs and the value in filters[c][f] 
            is a number between 0 and 1 that represents how sensitive this filter is to 
            that frequency.
    """
     
    assert min(freqs)<=min(cfreqs) and max(freqs)>=max(cfreqs), "cfreqs range must be within freqs range."

    filters = np.zeros((len(cfreqs), len(freqs)))
    
    for c in range(0,len(cfreqs)):
        if c==0: 
            # if we're on the first cfreq, use min(freq) for the low
            low_freq = min(freqs)
            high_freq = cfreqs[c+1]
        elif c==(len(cfreqs)-1):
            # if we're on the last cfreq, use max(freq) for the high
            low_freq = cfreqs[c-1]
            high_freq = max(freqs)
        else:
            low_freq = cfreqs[c-1]
            high_freq = cfreqs[c+1]
  
        for f in range(len(freqs)):
            
            if (freqs[f]> cfreqs[c]):
                x = freqs[f]
                b = -high_freq / (cfreqs[c]-high_freq) # offset
                a = 1./(cfreqs[c]-high_freq) # slope
                filters[c][f] = max([a*x+b,0.])
            elif (freqs[f]< cfreqs[c]):
                x = freqs[f]
                b = -low_freq / (cfreqs[c]-low_freq) # offset
                a = 1./(cfreqs[c]-low_freq) # slope
                filters[c][f] = max([a*x+b,0.])
            else:
                filters[c][f] = 1.0
    return filters

In [4]:
def single_mel_logspectrum(sgramslice,freqs, num_channels):
    """
    Builds a mel-frequency spectrogram. Computes the short term fourier transform of a 1-D numpy array 
    (the signal). It then creates a magnitude spectrogram of the signal and then remaps the energy num_channels
    frequency bins, spaced evenly in the log (or mel) frequency domain between the top key of the piano (4186 Hz) and 
    the bottom key of the piano (27.5 Hz). The vertical dimension is the index number of 
    the frequency channel. The horizontal dimension is time(in seconds), brightness corresponds to 
    amplitude (in dB). Only frequencies up to the top of the piano are displayed.
    
    Input Parameters
    ----------------
    signal: The 1-d (complex or real) numpy array containing the signal
    window_size: an integer scalar specifying the number of samples in a window
    hop_size: an integer specifying the number of samples between the start of adjacent windows
    sample_rate: an integer giving the sample rate of the input signal, in Hz
    num_channels: how many frequency bins to put the energy into
    channel_spacing: if 'mel', then space evenly in the mel range. Else, evenly in log2(freq)
            *NOTE* if you pick 88 channels and log2 spacing, this places a bin at each piano key
    display: a bool. If set to True, it plots the spectrogram. Else it does not.
    
    Returns
    -------
    an output tuple with 3 items
    
    lgram: a 2-D numpy array of real-valued numbers that contains the magnitude spectrogram.
           lgram[t,f] is the magnitude at time t and frequency f. This only contains values
           up to the pitch note of the piano (4186 Hz)
    times: a 1-D numpy array of non-negative real-values that gives the times,  
           times[t] gives the start time of the tth window in seconds
    freqs: a 1-D numpy array  of non-negative real values. freqs[f] gives the fth
           frequency bin in Hz. These are spaced evenly in the log2 of the frequency in Hz,
           with 12 to an octave. 
           
    Calling Example
    ---------------
    lgram,times,freqs = log_spectrogram(signal, window_size, hop_size, sample_rate, num_channels )

    """ 
    # your code goes here
    bottom_piano_note = 27.5  
    
    # get cfreqs, make filter
    lgramslice = np.zeros(num_channels)
    melfreqmin = freq2mel(27.5)
    melfreqmax = freq2mel(4186)
    melfreqs = np.linspace(melfreqmin, melfreqmax, num_channels)
    cfreqs = (10**(melfreqs/2595.0)-1)*700
    filters = make_triangular_filters(cfreqs, freqs)
    
    # use filter
    for f in range(len(cfreqs)):
        lgramslice[f] = np.sum(sgramslice*filters[f,:])
    
    # transform to dB
    lgramslice = 20 * np.log10(lgramslice)
    
    #plt.figure()
    #plt.plot(lgramslice)
    #plt.title('lgramslice')
    
    return lgramslice

In [5]:
from scipy.interpolate import interp1d
def detone(sgram_slice):
    length = len(sgram_slice)
    #plt.figure();plt.plot(sgram_slice)
    #print 'len(sgram_slice):', length
    index = np.arange(length)
    max_value = np.max(sgram_slice)
    for fundamental_guess in range(length):
        if sgram_slice[fundamental_guess] == max_value:
            break
    
    fundamental = 0
    for i in range(10,1,-1):
        for j in range(1,i):
            if sgram_slice[int(round(fundamental_guess*1.0/i*j))] < max_value*0.1:
                break
            fundamental = int(round(fundamental_guess*1.0/i))
            break
        if fundamental != 0:
            break
    if fundamental == 0:
        fundamental = fundamental_guess    
    
    f_l = fundamental-0.5
    f_r = fundamental+0.5
    harmonics_number = int(length/f_l)
    trash = []
    for num in range(1,harmonics_number):
        center_l = int(round(f_l*num))
        center_r = int(round(min(f_r*num, length)))
        if center_l == center_r:
            continue
        max_value = np.max(sgram_slice[center_l:center_r])
        for harmonic in range(center_l, center_r):
            if sgram_slice[harmonic] == max_value:
                break
        left = int(round(harmonic-f_l*0.1))
        right = int(round(harmonic+f_r*0.1)) 
        for _ in range(left, right+1):
            trash.append(_)    
    
    if 1203 in trash:
        trash2 = []
        for _ in trash:
            if _ < 1203:
                trash2.append(_)
        trash = trash2
        
    sgram_slice2 = np.delete(sgram_slice, trash)
    index2 = np.delete(index, trash)
    f = interp1d(index2, sgram_slice2, kind='cubic')
    
    index0 = index
    sgram_slice0 = sgram_slice
    sgram_slice = f(index)
    #plt.figure();plt.plot(sgram_slice)
    #print len(sgram_slice)
    return sgram_slice

In [6]:
 def getcepstrum(file, cgramdata):
    signal, sr = librosa.load(file, 22050)
    sample_rate = sr
    window_size = 2408
    hop_size = 1024
    sample_rate = 22050
    num_channels = 40
    sgram,times,freqs = spectrogram(signal, window_size, hop_size, sample_rate, display = False, tick_labels='frame-freqindex')

    rowsum = np.sum(sgram, axis = 1)
    #plt.figure;plt.plot(rowsum)  ##
    #plt.title('rowsum')
    max_value = max(rowsum)
    
    stastics = {}
    index = 0
    pointer1 = 0
    pointer2 = 0
    j = 0
    
    threshold = max_value*0.3   ######### threshold
    # print len(rowsum)
    # divide audio
    while j < len(times)-2:
        for i in range(j, len(times)-1):
            if (rowsum[i] < threshold and rowsum[i+1] > threshold) or (i == 0 and rowsum[i] > threshold):
                pointer1 = i+1
                for j in range(pointer1, len(times)-1):
                    if (rowsum[j] > threshold and rowsum[j+1] < threshold) or (j == len(times)-2 and rowsum[j] >threshold):
                        
                        pointer2 = j+1
                        stastics[index] = sgram[pointer1:pointer2, :]
                        #print pointer1, pointer2   #########
                        index = index + 1   
                        break            
                break
        if i == len(times)-2:
            break

    cgramslices = {}
    for i in range(len(stastics)):
        stastics_i = stastics[i]
        sgram_slice = np.sum(stastics_i, axis = 0)
        #plt.figure();plt.plot(sgram_slice)
        sgram_slice = detone(sgram_slice)     ########## detone function
        lgramslice = single_mel_logspectrum(sgram_slice,freqs, num_channels) ########## single_mel_logspectrum
        cgramslices[i] = sp.fftpack.dct(lgramslice, 1)
        #plt.figure()
        a = cgramslices[i]
        #a = cgramslices[i]/max(np.abs(cgramslices[i]))
        plt.plot(a[1:])
        #plt.figure();plt.plot(lgramslices[i])
    
    if cgramdata == {}:
        maxkey = -1
    else:
        maxkey = max(cgramdata)
    for _ in cgramslices:
        cgramdata[maxkey+1+_] = cgramslices[_]
        
    return cgramdata

cgramdata = {}
cgramdata = getcepstrum('bells.brass.C6B6.wav', cgramdata)   # 检测某一音频

In [ ]:
cgramdata = {}

import os
x = os.listdir(os.curdir)

for i in x:
    if ('Tuba' in i) and ('.wav' in i):              # 检测同一乐器的所有音频
        cgramdata = getcepstrum(i, cgramdata)

instrument = []
for key in cgramdata:
    A = list(cgramdata[key])
    instrument.append(A)

from tempfile import TemporaryFile
outfile = TemporaryFile()
np.savez("Tuba.npz", instrument)
npzfile = np.load("Tuba.npz")
a = npzfile['arr_0']
print len(a[0,:])
print len(a[:,0])